In [1]:
%config IPCompleter.greedy=True

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint



#Import API key
from api_keys import google_API_key

gmaps.configure(api_key=google_API_key)

In [2]:
csv_path = os.path.join("..","WeatherPy","output_data" , "cities.csv" )

weather_data = pd.read_csv(csv_path)

del weather_data["Unnamed: 0"]

weather_data.head()

,City,lat,Lng,Max Temp (F),Humidity,Cloudiness,Wind Speed,County,Date
0,busselton,-33.65,115.33,55.00,83,0,16.64,AU,1593984518
1,sao filipe,14.90,-24.50,73.94,82,71,11.95,CV,1593984519
2,hermanus,-34.42,19.23,51.01,95,100,2.71,ZA,1593984519
3,kodiak,57.79,-152.41,55.40,87,90,8.05,US,1593984519
4,cabo san lucas,22.89,-109.91,97.00,36,5,21.92,MX,1593984519


In [3]:
locations = weather_data[["lat", "Lng"]]
humidity = weather_data["Humidity"]

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)



fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
vaction_location = weather_data.loc[(weather_data["Max Temp (F)"] > 70) &
                                   (weather_data["Max Temp (F)"] < 80) &
                                    (weather_data["Wind Speed"] < 10) &
                                    (weather_data["Cloudiness"] == 0) &
                                   (weather_data["Humidity"]< 60) &
                                   (weather_data["Humidity"]> 40)]

vaction_location.head()

,City,lat,Lng,Max Temp (F),Humidity,Cloudiness,Wind Speed,County,Date
87,matveyev kurgan,47.56,38.87,77.32,41,0,3.18,RU,1593984528
150,cavalcante,-13.80,-47.46,71.85,47,0,5.41,BR,1593984535
437,sao miguel do araguaia,-13.28,-50.16,79.88,41,0,2.39,BR,1593984567
485,formosa do rio preto,-11.05,-45.19,76.21,45,0,4.00,BR,1593984572


In [5]:

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "keyword" : "hotel",
    "location": "", 
    "radius": "5000",
    "type": "lodging",
    "key": google_API_key}


hotel_name = []   
country = []
city = []
hotel_lat_lng = []


for i in range(len(vaction_location)):

    print("Getting data for" + str(vaction_location.iloc[i,0]))
    
    params['location'] = str(vaction_location.iloc[i, 1]) + ", " + str(vaction_location.iloc[i,2])

    response = requests.get(base_url, params=params).json()

    hotel_name.append(response['results'][0]["name"])
    city.append(response['results'][0]['plus_code']['compound_code'])
    hotel_lat_lng.append((float( response['results'][0]['geometry']['location']['lat'] ), 
                         float( response['results'][0]['geometry']['location']['lng'] )))
    
    


Getting data formatveyev kurgan
Getting data forcavalcante
Getting data forsao miguel do araguaia
Getting data forformosa do rio preto


In [13]:

marker = gmaps.marker_layer(hotel_lat_lng, hover_text =hotel_name)

fig.add_layer(marker)




fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
#locations = hotel_df[["Lat", "Lng"]]